In [1]:
using LightGraphs, Plots, GraphPlot, Compose, Random

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


In [2]:
include("ContactModels.jl")

rewire! (generic function with 1 method)

In [3]:
include("EpiSim.jl")

Main.EpiSim

## Load the data


In [4]:
using CSV

In [5]:
wapop=CSV.read("all-wa.csv")

,City / Town,Column2,Region,Remoteness Areas
,String⍰,String⍰,String,String
1,Perth,"1,874,578",Perth,Major Cities of Australia
2,missing,missing,Perth,Remote
3,Bunbury,"71,090",South West,Inner Regional Australia
4,missing,missing,South West,Outer Regional Australia
5,Geraldton,"31,982",Mid West,Inner Regional Australia
6,missing,missing,Mid West,Outer Regional Australia
7,Kalgoorlie-Boulder,"29,875",Goldfields-Esperance,Inner Regional Australia
8,missing,missing,Goldfields-Esperance,Outer Regional Australia
9,Albany,"29,373",Great Southern,Inner Regional Australia


## remove the empty towns

wapop2 is wapop with allt he empty towns deleted.  Both data frames come `all-wa.csv` which is a mash up of the two files that orlando provided.

In [6]:
(nr,nc)=size(wapop)
nr=nr-58
rowidx=[]
for i in 59:nr
    if popl[i]!=0
        rowidx=[rowidx; i]
    end
end
rowidx=[1:116; rowidx.+58]
wapop2=wapop[rowidx,:] #femove all empty settlements.

UndefVarError: UndefVarError: popl not defined

In [7]:
wapop=wapop2

UndefVarError: UndefVarError: wapop2 not defined

## Metric for similarity

`simiaritytown` computes a measure for the similarity between two locales - based on laguages spoken and whether they are within the same districts. Note that `majorlocalities` is hardcoded - I'm lazy.

In [8]:
function similaritytown(df,i,j)
    majorlocalities=58
    # similaritytown(df,i,j)
    # Define a "similarity" between two locations. This code is an ad-hoc rule that is an 
    # attempt to model the amount of movement between discrete regions. Dataframe df is 
    # assumed to be in the Orlando format as illustrated above.
    bonus1=100  #effectively, the larger the bonus, the more we include non-indigeneos travel.
    bonus2=10
    if i<j
        score=similaritytown(df,j,i)
        return score
    elseif i<=majorlocalities && j<=majorlocalities
        x1= convert(Matrix{Int}, df[2*i-1:2*i,6:end])
        x2= convert(Matrix{Int}, df[2*j-1:2*j,6:end])
        languagesimilarity=x1[:]'*x2[:]        #or not
        regionbonus= (df[2*i,3]==df[2*j,3])*bonus1
    elseif i>majorlocalities && j>majorlocalities
        x1= convert(Vector{Int}, df[majorlocalities+i,6:end])
        x2= convert(Vector{Int}, df[majorlocalities+j,6:end])
        languagesimilarity=x1[:]'*x2[:]        #or not
        regionbonus= (df[i+majorlocalities,3]==df[j+majorlocalities,3])*bonus1 #same region
        regionbonus= regionbonus + (df[i+majorlocalities,5]==df[j+majorlocalities,5])*bonus2 #and same locality
    else
        x1= convert(Vector{Int}, df[majorlocalities+i,6:end])
        x1= reshape(x1,1,length(x1))
        x2= convert(Matrix{Int}, df[2*j-1:2*j,6:end])
        languagesimilarity= (sum(x2,dims=1) * x1[:])[1]      #or not
        regionbonus= (df[i+majorlocalities,3]==df[2*j,3])*bonus1 #same region
    end
    score=Int(floor(sqrt(languagesimilarity+regionbonus))) # sqrt? why not?
    return score
end

similaritytown (generic function with 1 method)

In [9]:
dist=Array{Int,2}(undef,370,370)
for i in 1:370
    for j in 1:370
        dist[i,j]=similaritytown(wapop,i,j)
    end
end

In [ ]:
plot(dist[:].+1,yaxis=:log)

In [ ]:
using LightGraphs, GraphPlot, SimpleWeightedGraphs, Plots

# Compute and view the inferred connectivity structure

In [ ]:
# extract information from dataframe
#
#population
majorlocalities=58
brk=majorlocalities*2
townpop1=wapopint=parse.(Int, replace.(wapop[1:2:brk,2], r","=> ""))
townpop2=wapopint=parse.(Int, replace.(wapop[brk+1:end,2], r","=> ""))
townpop=[townpop1; townpop2]
#townpop=townpop[1:end-1]
#region and assign colouring
nodecolor = [colorant"lightseagreen", colorant"orange", colorant"red", colorant"green", colorant"yellow", colorant"brown", colorant"pink", colorant"purple", colorant"blue", colorant"violet", colorant"black"]
townlabel=[wapop[1:2:brk,3]; wapop[brk+1:end,3]]
#townlabel=townlabel[1:end-1]
regions=unique(townlabel)
regioni=Array{Int,1}(undef,370)
for i in 1:370
    regioni[i]=findall(x->x==1,townlabel[i].==regions)[1]
end
nodefillc = nodecolor[regioni]
#town names
townlabel=[wapop[1:2:brk,1]; wapop[brk+1:end,1]]
#townlabel=townlabel[1:end-1]

In [ ]:
gplot(Graph(dist.>2),nodefillc=nodefillc)

In [ ]:
gplot(Graph(dist.>2),nodelabel=townlabel,nodelabelsize=log.(townpop),nodesize=log.(townpop),nodefillc=nodefillc)

In [ ]:
plotly()


In [ ]:
histogram(regioni,rotation=90,xticks=(1.25:(length(regions)+0.25),regions))

## Construct the full individual-level connectivity structure for WA

Each node is a person.

In [ ]:
function buildstate(statedata,netbuilder)
    # buildstate(statedata, netbuilder)
    ##############################################################################
    #statedata is a csv table loaded from the structure /format above (AKA the Orlando Format)
    #netbuilder is the intra-locale network construction rule (a function)
    ##############################################################################
    #
    #extract the data from CSV table
    majorlocalities=58
    brk=majorlocalities*2
    locale=[statedata[1:2:brk,1]; statedata[brk+1:end,1]]
    townpop1=wapopint=parse.(Int, replace.(wapop[1:2:brk,2], r","=> ""))
    townpop2=wapopint=parse.(Int, replace.(wapop[brk+1:end,2], r","=> ""))
    popl=[townpop1; townpop2]
    npl=length(popl)
    net=SimpleGraph() #empty graph
    transit=Array{Int64,2}(undef,npl,npl) #number of transits beween locale[i] and locale[j]
    for (i,town) in enumerate(locale)
        println("Adding ",town," (population: ",popl[i],")")
        #add the intralocale links
        if popl[i]>0
            netadd=netbuilder(popl[i]) #use the preassigned method to add the new component
            net=blockdiag(net,netadd)
        end
        #compute the amount of transit
        for j in 1:(i-1) #number of transit between here and every other previous part
        transit[i,j] = Int(floor(similaritytown(statedata,i,j)))
            transit[j,i]=transit[i,j]
        end
        transit[i,i]=0
    end
    #add the inter-locale links too --- could do it in the same (previous) loops, but this is clearer (I think)
    rr=[0; cumsum(popl)]
    nedges_add = 0
    for i in 1:npl
        for j in (i+1):npl
            addlink=transit[i,j]
            addlink=minimum([addlink  popl[i]  popl[j]])
            if addlink>0
                edg1=rand(collect(rr[i]+1:rr[i+1]),addlink)
                edg2=rand(collect(rr[j]+1:rr[j+1]),addlink)
                for k in 1:addlink
                    add_edge!(net,edg1[k],edg2[k]) #this is grossly inefficient
                    nedges_add += 1
                end
            end
        end
    end
    #all done
    println(" $nedges_add edges added")
    return net, transit, locale, popl, nedges_add
end

## Run the simulations

Some examples are given. Note that `ContactModels.jl` has been modified and may need to be modified further. Still to do - more than five person households.

In [ ]:
#net, transit, locale, popl, distrc = buildstate(wapop[1:end-1,:], x -> covidsafe(nomassmix(x),0.5))
#net, transit, locale, popl, distrc = buildstate(wapop[1:end-1,:], x -> nomassmix(x))
net, transit, locale, popl, nedges_added = buildstate(wapop, x -> fullmixing(x))

In [ ]:
#'reasonable' parameters ################################
epiparam=Dict()
epiparam["p0"]=0.2 #a guess - tuned to match observed data 
epiparam["p2"]=1/12 #revised infection rate with distancing measure
epiparam["q"]=1/7 #"up to" two weeks
epiparam["r0"]=1/14 #about two weeks for mild, 3-6 for severe
epiparam["r2"]=1/4 #revised removal rate (now due to testing and isolation)
#########################################################
epiparam["nseeds"]=5 #probably too many, consider dropping. 
epiparam["pop"]=size(net)[1]

In [ ]:
ndays=90
nsims=200
St,Et,It,Rt = EpiSim.episim(net, epiparam, ndays, nsims)  #no change-point nsims simulations for ndays days

In [ ]:
#log scale plot of model growth in infection 
plot(title="Simulation across WA", ylabel="total infected",yaxis=:log)
EpiSim.plotquantiles(It+Rt.+1,:yellow,"90% CI",0.45) #need to add one, otherwise log gets upset
EpiSim.plotquantiles(It+Rt.+1,:red,"50% CI",0.25)

In [ ]:
#now, do the same things, but keep track of the count in each town
ndays=210
nsims=500
locpop=cumsum([1; popl],dims=1)
St,Et,It,Rt = EpiSim.episimcom(net, locpop, epiparam, ndays, nsims)  #no change-point nsims simulations for ndays days
#St[i,j,k] is the number of susceptibles in locale k on day i of simulation j
;


In [ ]:
#log scale plot of model growth in infection 
lcl=50
plot(title="Impact on "*locale[lcl], ylabel="total infected",yaxis=:log)
EpiSim.plotquantiles(It[:,:,lcl]+Rt[:,:,lcl].+1,:yellow,"90% CI",0.45) #need to add one, otherwise log gets upset
EpiSim.plotquantiles(It[:,:,lcl]+Rt[:,:,lcl].+1,:red,"50% CI",0.25)

## View results

In [ ]:
nday=[60 45 30 15]
impactloc=Array{Float64,2}(undef, length(locale), length(nday))
for n in 1:1:length(nday)
    for k in 1:length(locale)
        impactloc[k,n]=100*sum((It[nday[n],:,k]+Rt[nday[n],:,k]).>0)/nsims
       # println(locale[k]," affected ",impactloc[k],"%")
    end
end

In [ ]:
bar(locale,impactloc, orientation='x', rotation=90, xlabel="probability of local transmission", yticks=(0.5:1:length(locale),locale),linetype=:bar,label=nday,size=(600,8000),title="Estimated vulnerability" )


## Michael Small (21/6/20) 

Work in progress, for use as is only.